# Binary encoding

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, DepthwiseConv2D
import tensorflow as tf

In [2]:
import numpy as np

# Función para convertir binario a Gray code
def binary_to_gray(binary_str):
    return binary_str[0] + ''.join(str(int(binary_str[i-1]) ^ int(binary_str[i])) for i in range(1, len(binary_str)))

# Función para convertir Gray code a binario
def gray_to_binary(gray_str):
    binary_str = gray_str[0]
    for i in range(1, len(gray_str)):
        binary_str += str(int(binary_str[i - 1]) ^ int(gray_str[i]))
    return binary_str

# Diccionarios para codificación/decodificación
layer_type_options = {
    'Conv2D': '0000', 
    'BatchNorm': '0001', 
    'MaxPooling': '0010', 
    'Dropout': '0011', 
    'Dense': '0100', 
    'Flatten': '0101',
    'DepthwiseConv2D': '0110',  # Mantenemos DepthwiseConv2D
    'DontCare': '0111',  # Capa "don't care"
    'Repetition': '1***'  # Capa "Repetition"
}
filter_options = {32: '00', 30: '01', 16: '10', 8: '11'}
stride_options = {1: '0', 2: '1'}
dropout_options = {0.2: '00', 0.3: '01', 0.4: '10', 0.5: '11'}
neuron_options = {256: '00', 128: '01', 32: '10', 1: '11'}
activation_options = {'relu': '00', 'leaky_relu': '01', 'sigmoid': '10', 'tanh': '11'}

def encode_layer_params(layer_type, filters=None, strides=None, dropout=None, neurons=None, activation=None, repetition_layers=None, repetition_count=None):
    # Si es una capa de repetición
    if repetition_layers is not None and repetition_count is not None:
        repetition_layers_gray = binary_to_gray(f'{repetition_layers:03b}')  # 3 bits en Gray Code
        repetition_count_gray = binary_to_gray(f'{repetition_count:05b}')    # 5 bits en Gray Code
        binary_representation = '1' + repetition_layers_gray + repetition_count_gray
        return binary_representation

    # Capa normal
    binary_representation = layer_type_options.get(layer_type, '0000')
    
    if layer_type == 'Conv2D':
        binary_representation += filter_options.get(filters, '00')
        binary_representation += stride_options.get(strides, '0')
        binary_representation += activation_options.get(activation, '00')
    
    elif layer_type == 'DepthwiseConv2D':
        binary_representation += filter_options.get(filters, '00')
        binary_representation += stride_options.get(strides, '0')
        binary_representation += activation_options.get(activation, '00')

    elif layer_type == 'MaxPooling':
        binary_representation += stride_options.get(strides, '0')
    
    elif layer_type == 'Dropout':
        binary_representation += dropout_options.get(dropout, '00')
    
    elif layer_type == 'Dense':
        binary_representation += neuron_options.get(neurons, '00')
        binary_representation += activation_options.get(activation, '00')
    
    elif layer_type == 'BatchNorm':
        # BatchNorm no tiene parámetros adicionales
        binary_representation += '0000'  # Sin parámetros adicionales

    elif layer_type == 'Flatten':
        # Flatten no tiene parámetros adicionales
        binary_representation += '0000'  # Sin parámetros adicionales

    # Si no es una capa que requiere muchos parámetros, rellenamos a 9 bits
    binary_representation = binary_representation.ljust(9, '0')  # Asegurarse de llenar hasta 9 bits
    return binary_representation


# Función para decodificar parámetros de las capas
def decode_layer_params(encoded_binary):
    if len(encoded_binary) < 9:  # Ahora verificamos si el string tiene al menos 9 bits
        print(f"Error: The encoded layer binary {encoded_binary} is too short!")
        return {'type': 'DontCare'}  # Usar una capa "DontCare" como valor por defecto
    
    if encoded_binary[0] == '1':  # Es una capa de repetición
        repetition_layers = gray_to_binary(encoded_binary[1:4])
        repetition_count = gray_to_binary(encoded_binary[4:])
        return {
            'type': 'Repetition',
            'repetition_layers': int(repetition_layers, 2),
            'repetition_count': int(repetition_count, 2)
        }
    
    # Si no es repetición, procesamos normalmente
    layer_type = encoded_binary[1:4]
    
    if layer_type == '000':  # Conv2D
        filters = decode_value(encoded_binary[4:6], filter_options, 32)
        strides = decode_value(encoded_binary[6], stride_options, 1)
        activation = decode_value(encoded_binary[7:], activation_options, 'relu')
        return {
            'type': 'Conv2D',
            'filters': filters,
            'strides': strides,
            'activation': activation
        }
    elif layer_type == '001':  # BatchNorm
        return {'type': 'BatchNorm'}
    
    elif layer_type == '010':  # MaxPooling
        strides = decode_value(encoded_binary[4], stride_options, 1)
        return {'type': 'MaxPooling', 'strides': strides}
    
    elif layer_type == '011':  # Dropout
        rate = decode_value(encoded_binary[4:6], dropout_options, 0.2)
        return {'type': 'Dropout', 'rate': rate}
    
    elif layer_type == '100':  # Dense
        neurons = decode_value(encoded_binary[4:6], neuron_options, 256)
        activation = decode_value(encoded_binary[6:], activation_options, 'relu')
        return {'type': 'Dense', 'units': neurons, 'activation': activation}
    
    elif layer_type == '101':  # Flatten
        return {'type': 'Flatten'}
    
    elif layer_type == '110':  # DepthwiseConv2D
        filters = decode_value(encoded_binary[4:6], filter_options, 32)
        strides = decode_value(encoded_binary[6], stride_options, 1)
        activation = decode_value(encoded_binary[7:], activation_options, 'relu')
        return {
            'type': 'DepthwiseConv2D',
            'filters': filters,
            'strides': strides,
            'activation': activation
        }
    
    elif layer_type == '111':  # Don't care (relleno)
        return {'type': "DontCare"}  # Se omite

    return None

# Función auxiliar para decodificar valores en función de los diccionarios
def decode_value(bits, options_dict, default_value):
    # Buscar los bits en los valores del diccionario y devolver la clave correspondiente
    for key, value in options_dict.items():
        if value == bits:
            return key
    # Si no se encuentra, devolver el valor por defecto
    return default_value

# Ejemplo de codificación y decodificación
# Codificación y decodificación de Conv2D
binary_conv2d = encode_layer_params('Conv2D', filters=32, strides=1, activation='relu')
decoded_conv2d = decode_layer_params(binary_conv2d)
print(f"\nBinary encoding of Conv2D: {binary_conv2d}")
print(f"Decoded Conv2D: {decoded_conv2d}")

# Codificación y decodificación de Dropout
binary_dropout = encode_layer_params('Dropout', dropout=0.3)
decoded_dropout = decode_layer_params(binary_dropout)
print(f"\nBinary encoding of Dropout: {binary_dropout}")
print(f"Decoded Dropout: {decoded_dropout}")

# Codificación y decodificación de Dense
binary_dense = encode_layer_params('Dense', neurons=128, activation='relu')
decoded_dense = decode_layer_params(binary_dense)
print(f"\nBinary encoding of Dense: {binary_dense}")
print(f"Decoded Dense: {decoded_dense}")

# Codificación y decodificación de una capa de repetición
binary_repetition = encode_layer_params(None, repetition_layers=3, repetition_count=5)
decoded_repetition = decode_layer_params(binary_repetition)
print(f"\nBinary encoding of Repetition: {binary_repetition}")
print(f"Decoded Repetition: {decoded_repetition}")



Binary encoding of Conv2D: 000000000
Decoded Conv2D: {'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}

Binary encoding of Dropout: 001101000
Decoded Dropout: {'type': 'Dropout', 'rate': 0.3}

Binary encoding of Dense: 010001000
Decoded Dense: {'type': 'Dense', 'units': 128, 'activation': 'relu'}

Binary encoding of Repetition: 101000111
Decoded Repetition: {'type': 'Repetition', 'repetition_layers': 3, 'repetition_count': 5}


# Binary Encoding of Building Blocks for Neural Network Architectures

This module implements the **binary encoding** of various building blocks that make up the architecture of a neural network. Each building block (layers like Conv2D, MaxPooling, Dense, etc.) is represented in binary format based on its key parameters (number of filters, kernel size, number of neurons, activations, etc.). This encoding is useful for optimization processes like **NSGA-III** and later conversion to **Gray code**.

### Function `encode_layer_params`

This function generates the binary representation of the parameters of a given layer, depending on the type of layer and its specific parameters.

### Function Parameters

- `layer_type`: (str) The type of layer. It can be one of the following:
  - `Conv2D`
  - `BatchNorm`
  - `MaxPooling`
  - `Dropout`
  - `Dense`
  - `Flatten`
  
- `filters`: (int) [Optional] Number of filters for `Conv2D` layers. Valid options: `32`, `30`, `16`, `8`.

- `kernel_size`: (tuple) [Optional] Kernel size for `Conv2D`. Currently, only `(3, 3)` is supported.

- `strides`: (int) [Optional] Stride used in `Conv2D` or `MaxPooling`. Valid options: `1` and `2`.

- `padding`: (str) [Optional] Padding type in `Conv2D`. Currently, only `'same'` is supported.

- `dropout`: (float) [Optional] Dropout rate for `Dropout` layers. Valid options: `0.2`, `0.3`, `0.5`.

- `neurons`: (int) [Optional] Number of neurons in `Dense` layers. Valid options: `256`, `128`, `32`.

- `activation`: (str) [Optional] Activation function for `Conv2D` or `Dense` layers. Valid options: `'ReLU'`, `'LeakyReLU'`, `'Sigmoid'`.

### Return

- `binary_representation`: (str) A string of bits representing the layer parameters in binary format.

## Layer Type Encoding

Each layer type is represented by a specific sequence of bits. The layer types and their corresponding encodings are:

- **Conv2D**: `'000'`
- **BatchNorm**: `'001'`
- **MaxPooling**: `'010'`
- **Dropout**: `'011'`
- **Dense**: `'100'`
- **Flatten**: `'101'`

## Parameters Encoded by Layer Type

### Conv2D
The `Conv2D` layer includes the following encoded parameters:

- **Number of filters**:
  - `32`: `'00'`
  - `30`: `'01'`
  - `16`: `'10'`
  - `8`: `'11'`

- **Kernel size**:
  - `(3, 3)`: `'0'` (only this size is supported for now)

- **Stride**:
  - `1`: `'0'`
  - `2`: `'1'`

- **Padding**:
  - `'same'`: `'0'`

- **Activation function**:
  - `ReLU`: `'00'`
  - `LeakyReLU`: `'01'`

### BatchNorm
The `BatchNorm` layer has no additional parameters, only its layer type encoding (`'001'`).

### MaxPooling
The `MaxPooling` layer has a single encoded parameter:

- **Stride**:
  - `1`: `'0'`
  - `2`: `'1'`

### Dropout
The `Dropout` layer has the following encoded dropout rates:

- **Dropout Rate**:
  - `0.2`: `'00'`
  - `0.3`: `'01'`
  - `0.5`: `'10'`

### Dense
The `Dense` layer has the following encoded parameters:

- **Number of neurons**:
  - `256`: `'00'`
  - `128`: `'01'`
  - `32`: `'10'`

- **Activation function**:
  - `ReLU`: `'00'`
  - `LeakyReLU`: `'01'`
  - `Sigmoid`: `'10'`

### Flatten
The `Flatten` layer has no additional parameters, only its layer type encoding (`'101'`).

## Gray Code Conversion

In addition to binary encoding, this module supports **Gray code** conversion for efficient optimization, where only one bit changes between consecutive values.

### Function `binary_to_gray`

This function converts a binary string to Gray code using the following logic:
- The first bit in Gray code is the same as the first bit in binary.
- For each subsequent bit, the Gray code bit is the XOR of the current binary bit and the previous binary bit.

### Function Parameters

- `binary_str`: (str) A string representing a binary number.

### Return

- `gray_str`: (str) A string representing the corresponding Gray code.

### Function `gray_to_binary`

This function converts a Gray code string back to its binary equivalent using the following logic:
- The first bit of the binary string is the same as the first bit of the Gray code.
- For each subsequent bit, the binary bit is the XOR of the previous binary bit and the current Gray code bit.

### Function Parameters

- `gray_str`: (str) A string representing a Gray code number.

### Return

- `binary_str`: (str) A string representing the corresponding binary number.




In [3]:
# Función para eliminar campos adicionales como 'padding' y 'kernel_size'
def clean_decoded_model(model_dict):
    cleaned_layers = []
    
    for layer in model_dict['layers']:
        # Eliminar las capas 'DontCare' o de repetición
        if layer['type'] == 'DontCare' or layer['type'] == 'Repetition':
            continue  # No incluir la capa 'DontCare' o 'Repetition'
        
        # Remover 'kernel_size' y 'padding' si están presentes
        if 'kernel_size' in layer:
            del layer['kernel_size']
        if 'padding' in layer:
            del layer['padding']
        
        cleaned_layers.append(layer)
    
    return {'layers': cleaned_layers}


def fixArch(model_dict):
    """
    Corrige y valida la arquitectura del modelo, asegurando que el orden de las capas
    sea coherente y las capas Flatten no aparezcan antes de capas convolucionales o MaxPooling.
    También se aseguran las capas Flatten antes de Dense.
    """
    fixed_layers = []
    input_is_flattened = False  
    conv_or_pool_found = False  

    for layer in model_dict['layers']:
        # Si encontramos una capa de repetición
        if layer['type'] == 'Repetition':
            fixed_layers.extend(fixed_layers[-layer['repetition_layers']:] * layer['repetition_count'])
            continue  # Saltar a la siguiente capa después de aplicar la repetición

        # Si encontramos una capa Flatten después de Conv2D, DepthwiseConv2D o MaxPooling
        if layer['type'] == 'Flatten':
            if conv_or_pool_found:
                input_is_flattened = True
            else:
                print(f"Invalid placement: Replacing Flatten with DontCare.")
                fixed_layers.append({'type': 'DontCare'})
                continue
        
        if input_is_flattened and layer['type'] in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling']:
            print(f"Invalid placement: Replacing {layer['type']} with DontCare after Flatten.")
            layer = {'type': 'DontCare'}
        
        if layer['type'] == 'Dense' and not input_is_flattened:
            print(f"Inserting Flatten before Dense layer.")
            fixed_layers.append({'type': 'Flatten'})
            input_is_flattened = True 
        
        if layer['type'] in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling']:
            conv_or_pool_found = True

        fixed_layers.append(layer)

    model_dict['layers'] = fixed_layers
    return model_dict


def decode_model_architecture(encoded_string_binary, gene_length=9):
    model_dict = {'layers': []}
    
    index = 0
    while index < len(encoded_string_binary):
        # Extraer los próximos 9 bits de la codificación binaria
        binary_code_for_layer = encoded_string_binary[index:index+gene_length]
        
        # Decodificar una capa completa usando decode_layer_params
        decoded_layer = decode_layer_params(binary_code_for_layer)
        
        # Añadir la capa decodificada al modelo
        model_dict['layers'].append(decoded_layer)
        
        # Avanzar el índice por el tamaño del gen (9 bits)
        index += gene_length
    
    return fixArch(model_dict)




def encode_model_architecture(model_dict, max_alleles=108):
    encoded_layers_bin = []
    total_alleles = 0
    
    for layer in model_dict['layers']:
        if layer['type'] == 'Repetition':  # Codificar repetición de capas
            binary_encoding = encode_layer_params(
                layer_type='Repetition',
                repetition_layers=layer.get('repetition_layers'),
                repetition_count=layer.get('repetition_count')
            )
        else:
            binary_encoding = encode_layer_params(
                layer_type=layer['type'],
                filters=layer.get('filters'),
                strides=layer.get('strides'),
                dropout=layer.get('rate'),
                neurons=layer.get('units'),
                activation=layer.get('activation', '').lower()
            )
        
        if len(binary_encoding) != 9:  # Asegúrate de que cada capa esté correctamente codificada en 9 bits
            print(f"Warning: Layer {layer['type']} is not encoded in 9 bits: {binary_encoding}")
        
        total_alleles += len(binary_encoding)
        encoded_layers_bin.append(binary_encoding)
    
    # Verificar la longitud final y si necesita relleno
    while total_alleles < max_alleles:
        dont_care_encoding = encode_layer_params('DontCare')
        encoded_layers_bin.append(dont_care_encoding)
        total_alleles += len(dont_care_encoding)

    final_encoding_bin = ''.join(encoded_layers_bin[:max_alleles])
    print(f"Final Binary Encoding: {final_encoding_bin}")
    
    return final_encoding_bin


def verify_model_architecture(original_model):
    encoded_model = encode_model_architecture(original_model)
    print(f"Encoded Model: {encoded_model} (Length: {len(encoded_model)})")
    
    decoded_model = decode_model_architecture(encoded_model, gene_length=9)
    print("\nDecoded Model:")
    print(decoded_model)
    
    fixed_model = fixArch(decoded_model)
    print("\nFixed Model:")
    print(fixed_model)
    
    decoded_model_cleaned = clean_decoded_model(fixed_model)
    
    if original_model == decoded_model_cleaned:
        print("Modelo original y decodificado son iguales.")
        return True
    else:
        print("Original Model:")
        print(original_model)
        print("\nDecoded Model (cleaned):")
        print(decoded_model_cleaned)
        return False



# Ejemplo de uso
model_example = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}

# Verificar el modelo
print(verify_model_architecture(model_example))

# Construcción del modelo en TensorFlow desde el diccionario decodificado

class DontCareLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(DontCareLayer, self).__init__()

    def call(self, inputs):
        # Devuelve los inputs sin ninguna modificación (identidad)
        return inputs

def build_tf_model_from_dict(model_dict, input_shape=(28, 28, 3)):
    model = Sequential()
    previous_layers = []  # Lista para mantener las capas anteriores para repetirlas
    for layer in model_dict['layers']:
        if layer['type'] == 'Conv2D':
            conv_layer = Conv2D(filters=layer['filters'], 
                                kernel_size=(3, 3), 
                                strides=layer['strides'], 
                                padding='same', 
                                activation=layer['activation'])
            model.add(conv_layer)
            previous_layers.append(conv_layer)

        elif layer['type'] == 'DepthwiseConv2D':
            depthwise_layer = DepthwiseConv2D(kernel_size=(3, 3), 
                                              strides=layer['strides'], 
                                              padding='same', 
                                              activation=layer['activation'])
            model.add(depthwise_layer)
            previous_layers.append(depthwise_layer)

        elif layer['type'] == 'BatchNorm':
            batch_norm_layer = BatchNormalization()
            model.add(batch_norm_layer)
            previous_layers.append(batch_norm_layer)

        elif layer['type'] == 'MaxPooling':
            max_pool_layer = MaxPooling2D(pool_size=(2, 2), 
                                          strides=layer['strides'], 
                                          padding='same')
            model.add(max_pool_layer)
            previous_layers.append(max_pool_layer)

        elif layer['type'] == 'Flatten':
            flatten_layer = Flatten()
            model.add(flatten_layer)
            previous_layers.append(flatten_layer)

        elif layer['type'] == 'Dense':
            dense_layer = Dense(units=layer['units'], 
                                activation=layer['activation'])
            model.add(dense_layer)
            previous_layers.append(dense_layer)

        elif layer['type'] == 'Dropout':
            dropout_layer = Dropout(rate=layer['rate'])
            model.add(dropout_layer)
            previous_layers.append(dropout_layer)

        elif layer['type'] == 'Repetition':
            # Repetir las últimas 'repetition_layers' capas 'repetition_count' veces
            if len(previous_layers) >= layer['repetition_layers']:
                layers_to_repeat = previous_layers[-layer['repetition_layers']:]
                for _ in range(layer['repetition_count']):
                    for repeated_layer in layers_to_repeat:
                        # Clonamos la capa repetida usando `type` y los mismos parámetros
                        if isinstance(repeated_layer, Conv2D):
                            model.add(Conv2D(repeated_layer.filters, 
                                             kernel_size=(3, 3), 
                                             strides=repeated_layer.strides, 
                                             padding='same', 
                                             activation=repeated_layer.activation))
                        elif isinstance(repeated_layer, DepthwiseConv2D):
                            model.add(DepthwiseConv2D(kernel_size=(3, 3), 
                                                      strides=repeated_layer.strides, 
                                                      padding='same', 
                                                      activation=repeated_layer.activation))
                        elif isinstance(repeated_layer, BatchNormalization):
                            model.add(BatchNormalization())
                        elif isinstance(repeated_layer, MaxPooling2D):
                            model.add(MaxPooling2D(pool_size=(2, 2), 
                                                   strides=repeated_layer.strides, 
                                                   padding='same'))
                        elif isinstance(repeated_layer, Flatten):
                            model.add(Flatten())
                        elif isinstance(repeated_layer, Dense):
                            model.add(Dense(repeated_layer.units, 
                                            activation=repeated_layer.activation))
                        elif isinstance(repeated_layer, Dropout):
                            model.add(Dropout(repeated_layer.rate))
            else:
                print(f"Error: No hay suficientes capas para repetir {layer['repetition_layers']} veces.")
                continue

        elif layer['type'] == 'DontCare':
            # Capa DontCare se omite
            continue

    model.build(input_shape=(None,) + input_shape)

    return model



Final Binary Encoding: 010100000000000000000100000001010000010100000010000000001111000010011100011100000011100000011100000011100000
Encoded Model: 010100000000000000000100000001010000010100000010000000001111000010011100011100000011100000011100000011100000 (Length: 108)
Invalid placement: Replacing Flatten with DontCare.

Decoded Model:
{'layers': [{'type': 'DontCare'}, {'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'BatchNorm'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Flatten'}, {'type': 'Dense', 'units': 256, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.5}, {'type': 'Dense', 'units': 1, 'activation': 'relu'}, {'type': 'DontCare'}, {'type': 'DontCare'}, {'type': 'DontCare'}, {'type': 'DontCare'}]}

Fixed Model:
{'layers': [{'type': 'DontCare'}, {'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'BatchNorm'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Flatten'}, {'type': 'Dense', 'units': 256, 'activation

In [4]:
# Ejemplo 1: build_CNN_LF_model
model_CNN_LF = {
    "layers": [
        {"type": "Conv2D", "filters": 30, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},  # Revisar 'filters'
        {"type": "Dropout", "rate": 0.2},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.3},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 2: build_reduced_model
model_reduced = {
    "layers": [
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "leaky_relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 3: build_Spectro_CNN_model
model_spectro_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "MaxPooling", "strides": 2}, 
         
         
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
         {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        
    {"type":"Flatten"},
    {"type":"Dense","units":256,"activation":"relu"},   
    {"type":"Dropout","rate":0.5},
                
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo corregido: build_Spectro_CNN_model con capas de repetición
model_spectro_CNN_with_repetition = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "leaky_relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2}, 
        # Aquí indicamos que las siguientes 10 capas (5 Conv2D + 5 BatchNorm) se repiten 10 veces
        {"type": "Repetition", "repetition_layers": 3, "repetition_count": 31},
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}


# Ejemplo 4: Simple Conv2D model
model_simple_conv = {
    "layers": [
        {"type": "Conv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 5: Simple Dense model
model_dense_only = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 256, "activation": "relu"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dense", "units": 256, "activation": "sigmoid"}
    ]
}

# Ejemplo 6: Small CNN model with Dropout
model_small_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 8, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.2},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 7: Deep CNN model
model_deep_CNN = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Flatten"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 10, "activation": "sigmoid"}  # Revisar 'units'
    ]
}

# Ejemplo 8: Basic Dense with Dropout
model_dense_dropout = {
    "layers": [
        {"type": "Flatten"},
        {"type": "Dense", "units": 128, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Dense", "units": 32, "activation": "tanh"}
    ]
}

model_with_depthwise = {
    "layers": [
        {"type": "DepthwiseConv2D", "filters": 16, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "MaxPooling", "strides": 2},
        {"type": "Dense", "units": 128, "activation": "sigmoid"},
        {"type": "Dense", "units": 1, "activation": "tanh"}
    ]
}
model_with_global_avg_pooling = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "BatchNorm"},
        {"type": "GlobalAveragePooling2D"},
        {"type": "Dense", "units": 32, "activation": "relu"},
        {"type": "Dense", "units": 1, "activation": "sigmoid"}
    ]
}


In [5]:
# Función para validar si los valores del modelo son válidos
def validate_model_layers(model):
    # Verificar que el número de capas no exceda 12
    max_layers = 12
    if len(model['layers']) > max_layers:
        print(f"Invalid model: Exceeds maximum allowed layers ({max_layers})")
        return False
    
    consecutive_repetition_count = 0  # Contador de repeticiones consecutivas

    for i, layer in enumerate(model['layers']):
        layer_type = layer['type']
        
        if layer_type == 'Conv2D':
            filters = layer.get('filters')
            strides = layer.get('strides')
            activation = layer.get('activation')

            if filters not in filter_options:
                print(f"Invalid filters value in Conv2D layer: {filters}")
                return False
            if strides not in stride_options:
                print(f"Invalid strides value in Conv2D layer: {strides}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in Conv2D layer: {activation}")
                return False
        
        elif layer_type == 'Dense':
            units = layer.get('units')
            activation = layer.get('activation')

            if units not in neuron_options:
                print(f"Invalid units value in Dense layer: {units}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in Dense layer: {activation}")
                return False

        elif layer_type == 'Dropout':
            rate = layer.get('rate')
            if rate not in dropout_options:
                print(f"Invalid dropout rate value: {rate}")
                return False
        
        elif layer_type == 'MaxPooling':
            strides = layer.get('strides')
            if strides not in stride_options:
                print(f"Invalid strides value in MaxPooling layer: {strides}")
                return False
        
        elif layer_type == 'Flatten':
            # No se necesitan validaciones adicionales para Flatten
            pass

        elif layer_type == 'BatchNorm':
            # No se necesitan validaciones adicionales para BatchNorm
            pass

        elif layer_type == 'DepthwiseConv2D':
            filters = layer.get('filters')
            strides = layer.get('strides')
            activation = layer.get('activation')

            if filters not in filter_options:
                print(f"Invalid filters value in DepthwiseConv2D layer: {filters}")
                return False
            if strides not in stride_options:
                print(f"Invalid strides value in DepthwiseConv2D layer: {strides}")
                return False
            if activation not in activation_options:
                print(f"Invalid activation value in DepthwiseConv2D layer: {activation}")
                return False

        elif layer_type == 'Repetition':
            # Validamos las capas de repetición
            repetition_layers = layer.get('repetition_layers')
            repetition_count = layer.get('repetition_count')

            if not (1 <= repetition_layers <= 7):  # Codificamos 3 bits para capas a repetir
                print(f"Invalid repetition_layers value: {repetition_layers}")
                return False
            if not (1 <= repetition_count <= 31):  # Usamos 5 bits para el número de repeticiones
                print(f"Invalid repetition_count value: {repetition_count}")
                return False

            # Verificar si hay repeticiones consecutivas
            consecutive_repetition_count += 1
            if consecutive_repetition_count > 1:
                print(f"Invalid: Consecutive Repetition layers at Gen {i+1}.")
                return False

            # Verificar si el rango de repetición incluye otras capas de repetición
            start_index = max(0, i - repetition_layers)
            for j in range(start_index, i):
                if model['layers'][j]['type'] == 'Repetition':
                    print(f"Invalid: Repetition at Gen {i+1} includes another Repetition layer.")
                    return False
        else:
            consecutive_repetition_count = 0  # Reiniciar si no es capa de repetición

        # Verificar si el tipo de capa es válido
        if layer_type not in layer_type_options:
            print(f"Invalid layer type: {layer_type}")
            return False
        
    return True




# Función para verificar y construir el modelo
def run_verification_and_build(model):
    if validate_model_layers(model):
        print("\nModel is valid. Running verification and building the model:")
        
        # Verificación del modelo
        if verify_model_architecture(model):
            print("Model verification passed.")
        else:
            print("Model verification failed.")
        
        # Codificar y decodificar el modelo utilizando el nuevo esquema de 9 bits
        encoded_model = encode_model_architecture(model)
        if(len(encoded_model) > 108):
            print("Model is NOT correctly encoded.")
            return("Model is NOT correctly encoded.")
            
        decoded_model_dict = decode_model_architecture(encoded_model, gene_length=9)  # Cambiado a 9 bits
        
        # Construcción del modelo en TensorFlow desde el diccionario decodificado
        tf_model = build_tf_model_from_dict(decoded_model_dict)
        
        # Mostrar la estructura del modelo de TensorFlow
        try:
            tf_model.summary()
        except Exception as e:
            print(f"Error building model: {e}")
    else:
        print("Model is invalid. Skipping verification and build.")
    print("\n")


In [6]:

# Verificación de los modelos
print("Verifications:")

print("\nModel 1: CNN_LF")
run_verification_and_build(model_CNN_LF)  # Ejemplo 1

print("\nModel 2: Reduced")
run_verification_and_build(model_reduced)  # Ejemplo 2

print("\nModel 3: Spectro CNN")
run_verification_and_build(model_spectro_CNN)  # Ejemplo 3

print("\nModel 3.5 Spectro CNN with repetition")
run_verification_and_build(model_spectro_CNN_with_repetition)  # Ejemplo 3

print("\nModel 4: Simple Conv2D")
run_verification_and_build(model_simple_conv)  # Ejemplo 4

print("\nModel 5: Dense Only")
run_verification_and_build(model_dense_only)  # Ejemplo 5

print("\nModel 6: Small CNN")
run_verification_and_build(model_small_CNN)  # Ejemplo 6

print("\nModel 7: Deep CNN")
run_verification_and_build(model_deep_CNN)  # Ejemplo 7

print("\nModel 8: Dense with Dropout")
run_verification_and_build(model_dense_dropout)  # Ejemplo 8

print("\nModel with DepthwiseConv2D")
run_verification_and_build(model_with_depthwise)

print("\nModel with GlobalAveragePooling2D")
run_verification_and_build(model_with_global_avg_pooling)

Verifications:

Model 1: CNN_LF

Model is valid. Running verification and building the model:
Final Binary Encoding: 000001000001100000000100000001010000000010000001100000000100000001010000010100000010000000001101000010011100
Encoded Model: 000001000001100000000100000001010000000010000001100000000100000001010000010100000010000000001101000010011100 (Length: 108)

Decoded Model:
{'layers': [{'type': 'Conv2D', 'filters': 30, 'strides': 1, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.2}, {'type': 'BatchNorm'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Conv2D', 'filters': 16, 'strides': 1, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.2}, {'type': 'BatchNorm'}, {'type': 'MaxPooling', 'strides': 2}, {'type': 'Flatten'}, {'type': 'Dense', 'units': 256, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.3}, {'type': 'Dense', 'units': 1, 'activation': 'relu'}]}

Fixed Model:
{'layers': [{'type': 'Conv2D', 'filters': 30, 'strides': 1, 'activation': 'relu'}, {'type': 'Dropo

# Testing random generated architectures


In [7]:
# Función de validación según las reglas de construcción de la red
def validate_model_architecture(model_dict):
    """
    Valida que el modelo siga las reglas:
    1. Las primeras 3 capas deben ser Conv2D, DepthwiseConv2D, MaxPooling, BatchNorm o Dropout.
    2. Después de Flatten, solo se permiten capas Dense, BatchNorm, Flatten o Dropout.
    """
    conv_count = 0
    input_is_flattened = False  # Bandera para verificar si las entradas están aplanadas

    for i, layer in enumerate(model_dict['layers']):
        layer_type = layer['type']

        # Regla 1: Las primeras 3 capas deben ser de ciertos tipos
        if i < 3:
            if layer_type not in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling', 'BatchNorm', 'Dropout']:
                print(f"Invalid first 3 layers at position {i}: {layer_type} is not allowed.")
                return False  # Si las primeras capas no son válidas, retornamos False
        # Regla 2: Después de Flatten, solo ciertos tipos de capas están permitidos
        if input_is_flattened:
            if layer_type not in ['Dense', 'BatchNorm', 'Flatten', 'Dropout']:
                print(f"Invalid layer after Flatten at position {i}: {layer_type} is not allowed.")
                return False

        # Detectar si hay un Flatten para aplicar la segunda regla
        if layer_type == 'Flatten':
            input_is_flattened = True
    
    return True  # Si pasa todas las validaciones, es válido

# Función de reparación del modelo
def repair_model_architecture(model_dict):
    """
    Repara el modelo si no cumple con las reglas de construcción:
    1. Asegura que las primeras 3 capas sean de los tipos permitidos.
    2. Asegura que después de Flatten, solo existan capas Dense, BatchNorm, Flatten o Dropout.
    """
    conv_count = 0
    input_is_flattened = False  # Bandera para verificar si las entradas están aplanadas

    # Regla 1: Corregir las primeras 3 capas si no son válidas
    for i in range(min(3, len(model_dict['layers']))):
        layer_type = model_dict['layers'][i]['type']
        if layer_type not in ['Conv2D', 'DepthwiseConv2D', 'MaxPooling', 'BatchNorm', 'Dropout']:
            print(f"Repairing invalid layer at position {i}. Replacing {layer_type} with Conv2D.")
            model_dict['layers'][i] = {
                'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'
            }  # Reemplazamos por una capa Conv2D básica

    # Regla 2: Reemplazar capas inválidas después de Flatten
    for i in range(3, len(model_dict['layers'])):
        layer_type = model_dict['layers'][i]['type']

        # Si encontramos Flatten, aplicamos las restricciones
        if layer_type == 'Flatten':
            input_is_flattened = True
        elif input_is_flattened and layer_type not in ['Dense', 'BatchNorm', 'Flatten', 'Dropout']:
            print(f"Repairing invalid layer after Flatten at position {i}. Replacing {layer_type} with Dense.")
            model_dict['layers'][i] = {
                'type': 'Dense', 'units': 128, 'activation': 'relu'
            }  # Reemplazamos por una capa Dense básica

    return model_dict  # Devolver el modelo reparado

# Función para ejecutar validación y reparación
def validate_and_repair_model(model_dict):
    if validate_model_architecture(model_dict):
        print("Model is valid.")
    else:
        print("Model is invalid. Repairing model...")
        repaired_model = repair_model_architecture(model_dict)
        print("Model has been repaired.")
        return repaired_model  # Devuelve el modelo reparado

# Ejemplo de uso
model_example = {
    "layers": [
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},
        {"type": "Dropout", "rate": 0.5},
        {"type": "Flatten"},
        {"type": "Conv2D", "filters": 32, "strides": 1, "activation": "relu"},  # Capa inválida después de Flatten
        {"type": "Dense", "units": 256, "activation": "relu"}
    ]
}

# Validar y reparar si es necesario
repaired_model = validate_and_repair_model(model_example)
print("\nRepaired Model:")
print(repaired_model)


Invalid first 3 layers at position 2: Flatten is not allowed.
Model is invalid. Repairing model...
Repairing invalid layer at position 2. Replacing Flatten with Conv2D.
Model has been repaired.

Repaired Model:
{'layers': [{'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'Dropout', 'rate': 0.5}, {'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'Conv2D', 'filters': 32, 'strides': 1, 'activation': 'relu'}, {'type': 'Dense', 'units': 256, 'activation': 'relu'}]}


In [8]:
import random
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models

# Función para generar un string binario aleatorio de 108 alelos
def generate_random_binary_model():
    return ''.join(random.choice(['0', '1']) for _ in range(108))

# Función para imprimir el modelo en binario, con una capa por línea antes de decodificar
def print_model_layers_in_binary(binary_model, title="Arquitectura del Modelo (genes en binario):"):
    print(title)
    for index in range(0, len(binary_model), 9):
        gene = binary_model[index:index+9]
        layer_type = "Unknown"
        if gene[0] == '1':
            layer_type = "Repetition"
        else:
            binary_layer_type = gene[1:4]
            layer_type = {
                '000': 'Conv2D',
                '001': 'BatchNorm',
                '010': 'MaxPooling',
                '011': 'Dropout',
                '100': 'Dense',
                '101': 'Flatten',
                '110': 'DepthwiseConv2D',
                '111': 'DontCare'
            }.get(binary_layer_type, 'Unknown')
        print(f"Gen {index//9 + 1}: {layer_type} {gene}")
    print("\n")

# Función para imprimir la arquitectura del modelo con repetición agrupada
def print_decoded_architecture_grouped(genes):
    decoded_layers = []
    for gene in genes:
        layer_type = "Unknown"
        params = gene[4:]
        if gene[0] == '1':
            layer_type = "Repetition"
            params = gene[1:4]
        else:
            binary_layer_type = gene[1:4]
            layer_type = {
                '000': 'Conv2D',
                '001': 'BatchNorm',
                '010': 'MaxPooling',
                '011': 'Dropout',
                '100': 'Dense',
                '101': 'Flatten',
                '110': 'DepthwiseConv2D',
                '111': 'DontCare'
            }.get(binary_layer_type, 'Unknown')
        decoded_layers.append((layer_type, gene, params))

    architecture_str = ""
    i = 0
    while i < len(decoded_layers):
        layer_type, gene, params = decoded_layers[i]
        if layer_type == "Repetition":
            repetition_count = int(gene[1:4], 2)
            if repetition_count == 0:
                architecture_str += f"Invalid Repetition({gene})\n"
            else:
                repeated_block = []
                for j in range(repetition_count):
                    if i - (j + 1) >= 0:
                        repeated_block.insert(0, decoded_layers[i - (j + 1)])
                architecture_str += f"["
                for layer_info in repeated_block:
                    layer, g, p = layer_info
                    architecture_str += f"{layer}({g}) "
                architecture_str = architecture_str.strip() + f"] x {repetition_count}\n"
        else:
            architecture_str += f"{layer_type}({gene})\n"
        i += 1

    print("Arquitectura Decodificada (Agrupada):")
    print(architecture_str)
    print("\n")

# Función para asegurar que el primer gen no sea Repetition
def repair_first_gene(genes):
    repair_log = []
    if genes[0][0] == '1':  # Si es Repetition, reemplazar con Conv2D
        print(f"Invalid: First gene cannot be Repetition. Replacing with Conv2D.")
        genes[0] = '000000000'  # Conv2D por defecto
        repair_log.append(f"Gen 1: Replaced Repetition with Conv2D.")
    return genes, repair_log

def repair_ending_genes(genes):
    repair_log = []
    
    # La penúltima capa debe ser Flatten
    if genes[-2][0] == '1' or genes[-2][1:4] != '101':  # Verificamos si la penúltima capa es Flatten
        print("Invalid: Penultimate layer must be Flatten. Replacing.")
        genes[-2] = '010100000'  # Flatten por defecto
        repair_log.append("Penultimate layer replaced with Flatten.")
    
    # La última capa debe ser Dense con 1 neurona
    if genes[-1][0] == '1' or genes[-1][1:4] != '100' or genes[-1][4:] != '00001':  # Verificamos si la última es Dense con 1 neurona
        print("Invalid: Last layer must be Dense with 1 unit. Replacing.")
        genes[-1] = '010000001'  # Dense con 1 neurona
        repair_log.append("Last layer replaced with Dense (units=1).")
    
    return genes, repair_log


# Función para reparar el cromosoma
def repair_chromosome(binary_model):
    genes = [binary_model[i:i+9] for i in range(0, len(binary_model), 9)]
    repair_log = []

    # Reparar el primer gen
    genes, first_gene_log = repair_first_gene(genes)
    repair_log.extend(first_gene_log)

    # Reparar los genes finales
    genes, ending_genes_log = repair_ending_genes(genes)
    repair_log.extend(ending_genes_log)

    # Evitar repeticiones consecutivas de capas de tipo Repetition
    for i in range(1, len(genes)):
        if genes[i][0] == '1' and genes[i-1][0] == '1':
            print(f"Invalid: Consecutive Repetition layers at positions {i} and {i+1}. Replacing gene at position {i+1} with DontCare.")
            genes[i] = '011100000'  # DontCare
            repair_log.append(f"Gen {i+1}: Replaced Repetition with DontCare due to consecutive Repetition layers.")

    # Asegurar que las primeras 3 capas sean válidas
    valid_initial_layer_types = ['000', '001', '010', '110']  # Conv2D, BatchNorm, MaxPooling, DepthwiseConv2D
    for i in [0, 1, 2]:  # Primeros 3 genes
        if genes[i][0] == '1' or genes[i][1:4] not in valid_initial_layer_types:
            print(f"Invalid: Gene {i+1} is invalid in initial position. Replacing with Conv2D.")
            genes[i] = '000000000'  # Conv2D por defecto
            repair_log.append(f"Gen {i+1}: Replaced invalid layer with Conv2D.")

    # Asegurar que Dense siempre esté precedido por Flatten
    i = 1
    while i < len(genes) - 2:
        current_gene = genes[i]
        prev_gene = genes[i - 1]
        current_layer_type = current_gene[1:4] if current_gene[0] == '0' else 'Repetition'
        prev_layer_type = prev_gene[1:4] if prev_gene[0] == '0' else 'Repetition'

        if current_layer_type == '100':  # Dense
            if prev_layer_type != '101':  # No está precedido por Flatten
                print(f"Invalid: Dense layer at position {i+1} not preceded by Flatten. Inserting Flatten layer.")
                # Insertar Flatten antes de la capa actual
                flatten_gene = '010100000'  # Flatten por defecto
                genes.insert(i, flatten_gene)
                repair_log.append(f"Gen {i+1}: Inserted Flatten layer before Dense.")
                i += 1  # Incrementar para evitar loop infinito
        elif prev_layer_type == '101' and current_layer_type in ['000', '001', '010', '110']:
            # Evitar que Flatten sea seguido por capas convolucionales
            print(f"Invalid: Flatten layer at position {i} followed by convolutional layer. Replacing layer at position {i+1} with DontCare.")
            genes[i] = '011100000'  # Reemplazar con DontCare
            repair_log.append(f"Gen {i+1}: Replaced invalid layer with DontCare after Flatten.")
        i += 1

    # Reparar capas de Repetition que causan conflictos
    for i in range(len(genes)):
        if genes[i][0] == '1':  # Es una capa de Repetition
            repetition_count = int(genes[i][1:4], 2)
            if repetition_count == 0:
                print(f"Invalid: Repetition count is zero at position {i+1}. Replacing with DontCare.")
                genes[i] = '011100000'  # DontCare
                repair_log.append(f"Gen {i+1}: Replaced Repetition with DontCare due to zero repetition count.")
                continue
            start_index = i - repetition_count
            if start_index < 0:
                print(f"Invalid: Repetition at position {i+1} refers to non-existent layers. Adjusting repetition count.")
                adjusted_count = i
                if adjusted_count == 0:
                    genes[i] = '011100000'  # DontCare
                    repair_log.append(f"Gen {i+1}: Replaced Repetition with DontCare due to invalid repetition count.")
                else:
                    # Ajustar el conteo de repetición en el gen
                    genes[i] = '1' + format(adjusted_count, '03b') + genes[i][4:]
                    repair_log.append(f"Gen {i+1}: Adjusted repetition count to {adjusted_count}.")

    # Asegurar que el cromosoma tenga 12 genes
    while len(genes) > 12:
        genes.pop()
        repair_log.append("Removed extra gene to maintain chromosome length of 12.")
    while len(genes) < 12:
        genes.append('011100000')  # Añadir DontCare
        repair_log.append("Added DontCare gene to maintain chromosome length of 12.")

    repaired_binary_model = ''.join(genes)
    return repaired_binary_model, repair_log




# Función auxiliar para agregar capas al modelo según el tipo de capa
def add_layer_to_model(model, layer_info):
    layer_type = layer_info['type']
    params = layer_info.get('params', '0000000')
    if layer_type == 'Conv2D':
        filters = int(params[:3], 2) + 1  # Al menos 1 filtro
        kernel_size = int(params[3:5], 2) + 1  # Tamaño de kernel mínimo 1
        model.add(layers.Conv2D(filters=filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same'))
    elif layer_type == 'BatchNorm':
        model.add(layers.BatchNormalization())
    elif layer_type == 'MaxPooling':
        pool_size = int(params[:2], 2) + 2  # Tamaño de pooling mínimo 2
        model.add(layers.MaxPooling2D(pool_size=(pool_size, pool_size)))
    elif layer_type == 'Dropout':
        rate = int(params[:3], 2) / 10.0  # Dropout rate entre 0.0 y 0.7
        model.add(layers.Dropout(rate=rate))
    elif layer_type == 'Dense':
        units = int(params[:5], 2) + 1  # Al menos 1 unidad
        model.add(layers.Dense(units=units, activation='relu'))
    elif layer_type == 'Flatten':
        model.add(layers.Flatten())
    elif layer_type == 'DepthwiseConv2D':
        kernel_size = int(params[:3], 2) + 1  # Tamaño de kernel mínimo 1
        model.add(layers.DepthwiseConv2D(kernel_size=(kernel_size, kernel_size), activation='relu', padding='same'))
    elif layer_type == 'DontCare':
        pass  # No hacer nada
    else:
        pass  # Capa desconocida o no soportada

# Función para generar, validar, reparar y construir modelos de TensorFlow
def generate_and_print_tf_models(num_models=5):
    generated_tf_models = []

    with tqdm(total=num_models, desc="Progreso general", unit="model") as pbar:
        for i in range(num_models):
            pbar.set_description(f"Generando modelo {i + 1}")
            random_model_binary = generate_random_binary_model()
            print(f"\nGenerated Binary Model {i + 1}: {' '.join([random_model_binary[j:j+9] for j in range(0, len(random_model_binary), 9)])}")
            print_model_layers_in_binary(random_model_binary)

            pbar.set_description(f"Reparando modelo {i + 1}")
            repaired_model_binary, repair_log = repair_chromosome(random_model_binary)
            print(f"Repaired Binary Model {i + 1}: {' '.join([repaired_model_binary[j:j+9] for j in range(0, len(repaired_model_binary), 9)])}")
            
            print_model_layers_in_binary(repaired_model_binary)
            print_decoded_architecture_grouped([repaired_model_binary[j:j+9] for j in range(0, len(repaired_model_binary), 9)])
            
            # Construir y mostrar el resumen del modelo de TensorFlow
            pbar.set_description(f"Construyendo modelo TensorFlow {i + 1}")
            try:
                decoded_model = decode_model_architecture(repaired_model_binary, gene_length=9)  # Decodificar el modelo reparado
                tf_model = build_tf_model_from_dict(decoded_model)
                generated_tf_models.append(tf_model)
                print(f"\nModel {i + 1} Summary:")
                tf_model.summary()
                print("\n" + "="*80 + "\n")
            except Exception as e:
                print(f"Error building TensorFlow model for Model {i + 1}: {e}")

            pbar.update(1)
            pbar.set_description(f"Progreso general")

    return generated_tf_models

# Llamada a la función para generar e imprimir modelos de TensorFlow
generate_and_print_tf_models(num_models=1)


Construyendo modelo TensorFlow 1:   0%|          | 0/1 [00:00<?, ?model/s]


Generated Binary Model 1: 000110001 001001110 111001001 111100100 010110111 100100111 110100001 000100010 000101100 001010001 100100000 000011110
Arquitectura del Modelo (genes en binario):
Gen 1: BatchNorm 000110001
Gen 2: MaxPooling 001001110
Gen 3: Repetition 111001001
Gen 4: Repetition 111100100
Gen 5: Flatten 010110111
Gen 6: Repetition 100100111
Gen 7: Repetition 110100001
Gen 8: BatchNorm 000100010
Gen 9: BatchNorm 000101100
Gen 10: MaxPooling 001010001
Gen 11: Repetition 100100000
Gen 12: Conv2D 000011110


Invalid: Penultimate layer must be Flatten. Replacing.
Invalid: Last layer must be Dense with 1 unit. Replacing.
Invalid: Consecutive Repetition layers at positions 3 and 4. Replacing gene at position 4 with DontCare.
Invalid: Consecutive Repetition layers at positions 6 and 7. Replacing gene at position 7 with DontCare.
Invalid: Gene 3 is invalid in initial position. Replacing with Conv2D.
Repaired Binary Model 1: 000110001 001001110 000000000 011100000 010110111 100100111

Progreso general: 100%|██████████| 1/1 [00:00<00:00, 13.83model/s]        

In [ ]:
import random

# Función para generar mutaciones en una arquitectura conocida
def mutate_architecture(binary_model, mutation_rate=0.1):
    mutated_model = list(binary_model)  # Convertimos el string a lista para mutarlo
    for i in range(len(mutated_model)):
        if random.random() < mutation_rate:  # Decidir si mutar este bit
            # Mutar el bit (0 -> 1, 1 -> 0)
            mutated_model[i] = '1' if mutated_model[i] == '0' else '0'
    return ''.join(mutated_model)

# Función para realizar crossover entre dos modelos
def crossover(parent1, parent2):
    # Elegir un punto de corte al azar
    crossover_point = random.randint(1, len(parent1) - 1)
    
    # Generar hijos mezclando los padres
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    
    return child1, child2

# Función para generar la población
def generate_population(known_architectures, population_size=100, mutation_rate=0.1):
    population = []

    # Paso 1: Añadir las arquitecturas conocidas a la población
    population.extend(known_architectures)

    # Paso 2: Generar el resto de la población
    while len(population) < population_size:
        # Seleccionar dos padres al azar de la población actual
        parent1 = random.choice(population)
        parent2 = random.choice(population)
        
        # Realizar crossover para generar nuevos hijos
        child1, child2 = crossover(parent1, parent2)
        
        # Aplicar mutaciones a los hijos
        child1 = mutate_architecture(child1, mutation_rate)
        child2 = mutate_architecture(child2, mutation_rate)
        
        # Añadir los hijos a la población
        population.append(child1)
        population.append(child2)

    # Asegurarse de que la población tiene el tamaño exacto
    return population[:population_size]

# Ejemplo de arquitecturas conocidas (en binario)
known_architectures = [
    '0000100001100000001000000101000000010000011000000010000001010000101000001000000001101000100111001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000',  # Ejemplo 1
    '0010000000010001001000000001100100100000101000001001001010011100111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000111000001110000011100000',  # Ejemplo 2
    '0000000100100000000000010101000000000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000100000001001000000000000110100000100000000111100010011100'  # Ejemplo 3
]

# Generar una población de 100 individuos
population = generate_population(known_architectures, population_size=100)

# Imprimir algunos individuos generados
for i, individual in enumerate(population):
    print(f"Individuo {i+1}: {individual}")
    decoded_chromosome = decode_model_architecture(individual)  # Decodificar y mostrar la arquitectura
    print(decoded_chromosome)
